# Question 1

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

##### Getting Wiki page's content using beautiful soup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(website_url,'lxml') #getting the lxml format from the website
print(soup.prettify()) #pinting to check the table class 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XeqLzApAMEoAAFPAAnQAAABU","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":929562264,"wgRevisionId":929562264,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

#### The table of post codes in the website is identified using the class and then the tr elements in the table are identified using the findAll method tocreate a list called namelist
#### Each row of the table is a string in nameList

In [8]:
My_table = soup.find('table',{'class':'wikitable sortable'}) #the find method is to used to locate the table 
trs = My_table.findAll('tr') # all the sub elements in the table which are the rows are identified using the findAll method
nameList = [i.text for i in trs] # looping through to make it into a list
nameList[0:2]

['\nPostcode\nBorough\nNeighbourhood\n', '\nM1A\nNot assigned\nNot assigned\n']

#### Looping through the list and splitting the strings into individual columns using the \n delimiter 

In [9]:
ls = []
for k in nameList:
    ls.append(k.split('\n')) # the string has a delimiter \n which is used to split it to differentiate the columns
ls[0:2]

[['', 'Postcode', 'Borough', 'Neighbourhood', ''],
 ['', 'M1A', 'Not assigned', 'Not assigned', '']]

##### This is then added to a data frame
##### The the blank columns are dropped off and column headers are correctly named

In [11]:
df_test = pd.DataFrame(ls) 
df_test.drop([0,4], axis = 1, inplace = True) # the first and last columns are blank columns created from the split 
#These are dropped off 

# the column headers are in the first row. This row is used to replace the column headers of the data frame 
df_neigh =pd.DataFrame(df_test.values[1:], columns = df_test.iloc[0] ) 
df_neigh.head(2)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


##### The rows with Borough as 'Not assigned' are dropped off 

In [12]:
df_n1 = df_neigh[df_neigh.iloc[:,1] != 'Not assigned'].copy()

##### Indexes of the rows with 'Not assigned ' in the Neighbourhood column are identified and replaced with the Borough name

In [ ]:
narows = df_n1[df_n1['Neighbourhood'] == 'Not assigned'].index
df_n1.loc[narows ,'Neighbourhood'] = df_n1['Borough']

##### The dataframe is now grouped by Postcode and Borough to aggregate the Neighbourhoods by a comma

In [14]:
df_n2 = df_n1.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index()
df_n2.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### The dataframe has 103 rows and 3 columns 

In [16]:
df_n2.shape

(103, 3)

# Question 2

#### Reading the CSV file to get the geo coordinates of the postal codes

In [19]:
df_latlong = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlong.shape

(103, 3)

#### Merging this data from the CSV file to the data frame from Question 1 i.e. the data frame on postal codes scraped from wikipedia

In [37]:
df_n2_ll = df_n2.merge(df_latlong, left_on='Postcode', right_on='Postal Code').drop('Postal Code', axis = 1) #Dropping the additional postal code column

#print the shape and the first 5 rows of the new data frame 
print(df_n2_ll.shape)
df_n2_ll.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Question 3

#### Importing needed dependencies and setting up the variables for use later

In [39]:
import requests 
import numpy as np
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # mapping library

CLIENT_ID = 'OWU5LYGCIFULB5FRPEBMPR5SFDBNMHWS4EBY5HSY0DY3TMX4' # your Foursquare ID
CLIENT_SECRET = 'MFCOPWOFDDXYN4FKURRCLAIZAPVPA3RPB3R2U3BVVY0TNZJH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

##### Creating a function for querying foursquare and extracting the venue details close to the requested latitude and longitude

In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Handle those post codes which have no results 
        if not results:
            print(name, " postcode returned no results so skipping addition ")
        else:
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Calling the function on the data frame created in question 2 i.e. the one which has the latitude/logitude details along with the Postal Code 

In [130]:
Torrono_venues = getNearbyVenues(names=df_n2_ll['Postcode'],
                                   latitudes=df_n2_ll['Latitude'],
                                   longitudes=df_n2_ll['Longitude'])
print(Torrono_venues.shape)
Torrono_venues.head()

M1X  postcode returned no results so skipping addition 
M2M  postcode returned no results so skipping addition 
M9A  postcode returned no results so skipping addition 
(2232, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


#### The venue categories are now dummy coded and appended with the post code column to get a data frame ready for clustering

In [86]:
# one hot encoding of the venue categories
Torrono_onehot = pd.get_dummies(Torrono_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
Torrono_onehot['Postcode'] = Torrono_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [Torrono_onehot.columns[-1]] + list(Torrono_onehot.columns[:-1])
Torrono_onehot = Torrono_onehot[fixed_columns]

Torrono_onehot.head()

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### The data frame is grouped by the post code to get the average frequency of a venue at that post code

In [87]:
Torrono_grouped = Torrono_onehot.groupby('Postcode').mean().reset_index()
Torrono_grouped.head(2)

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### The postcodes are now  put through K means clustering to cluster by the type and frequency of a venue  

In [119]:
# set number of clusters
kclusters = 4

Torrono_grouped_clustering = Torrono_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4 ).fit(Torrono_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))
kmeans.labels_

100


array([3, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1])

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [121]:
#function to sort venues in descending order of frequency
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = Torrono_grouped['Postcode']

for ind in np.arange(Torrono_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Torrono_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(100, 11)


,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Intersection,Electronics Store,Rental Car Location,Spa,Pizza Place,Mexican Restaurant,Breakfast Spot,Medical Center,Yoga Studio,Diner
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Discount Store,Dessert Shop


#### the clustering labels are added to the data

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Torrono_merged = df_n2_ll

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Torrono_merged = Torrono_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

Torrono_merged.dropna(subset=['Cluster Labels'], axis =0, inplace = True)

print(Torrono_merged.shape)
Torrono_merged.head()

(100, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1.0,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Intersection,Electronics Store,Rental Car Location,Spa,Pizza Place,Mexican Restaurant,Breakfast Spot,Medical Center,Yoga Studio,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Discount Store,Dessert Shop


#### Postcodes with Coffee Shops primarily are clustered to cluster 1, Playgrounds are in cluster 0 , Parks/Rivers to cluster 2

In [157]:
Torrono_merged.groupby(['Cluster Labels','1st Most Common Venue'], sort=True)['Postcode'].count()

Cluster Labels  1st Most Common Venue     
0.0             Playground                     1
                Restaurant                     1
1.0             Airport Service                1
                Bakery                         3
                Bar                            2
                Baseball Field                 1
                Breakfast Spot                 1
                Business Service               1
                Cafeteria                      1
                Café                           8
                Clothing Store                 2
                Coffee Shop                   18
                Construction & Landscaping     1
                Curling Ice                    1
                Discount Store                 1
                Fast Food Restaurant           2
                Filipino Restaurant            1
                Furniture / Home Store         1
                Garden                         1
                General En

#### Mapping the clusters on a map centered around Toronto

In [126]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Torrono_merged['Latitude'], Torrono_merged['Longitude'], Torrono_merged['Postcode'], Torrono_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.4).add_to(map_clusters)
       
map_clusters